In [ ]:
!pip3 install -q langchain==0.0.152

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 666.9/666.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip3 install -q pyllamacpp==1.0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.1/270.1 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.6 MB/s eta 0:00:00


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
local_path = './models/gpt4all-lora-quantized-ggml.bin'  # replace with your desired local file path

In [ ]:
import requests
from pathlib import Path
from tqdm import tqdm

local_path = './models/gpt4all-lora-quantized-ggml.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

# Example model. Check https://github.com/nomic-ai/pyllamacpp for the latest models.
url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'
# url = 'http://gpt4all.io/models/ggml-gpt4all-l13b-snoozy.bin'

# send a GET request to the URL to download the file. Stream since it's large
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response to it in chunks
# This is a large file, so be prepared to wait.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

514266it [01:37, 5285.36it/s]


https://github.com/ggerganov/llama.cpp#using-gpt4all

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!cd llama.cpp && git checkout 2b26469

Cloning into 'llama.cpp'...
remote: Enumerating objects: 4187, done.
remote: Counting objects: 100% (1748/1748), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 4187 (delta 1654), reused 1573 (delta 1553), pack-reused 2439
Receiving objects: 100% (4187/4187), 3.64 MiB | 15.72 MiB/s, done.
Resolving deltas: 100% (2836/2836), done.
Note: switching to '2b26469'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 2b26469 convert.py: Support models which are stored in a single pytorch_model.bin (

In [ ]:
!python3 llama.cpp/convert.py ./models/gpt4all-lora-quantized-ggml.bin

Loading model file models/gpt4all-lora-quantized-ggml.bin
Writing vocab...
[  1/291] Writing tensor tok_embeddings.weight                  | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  2/291] Writing tensor norm.weight                            | size   4096           | type UnquantizedDataType(name='F32')
[  3/291] Writing tensor output.weight                          | size  32001 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  4/291] Writing tensor layers.0.attention.wq.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  5/291] Writing tensor layers.0.attention.wk.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addends=False, have_g_idx=False)
[  6/291] Writing tensor layers.0.attention.wv.weight           | size   4096 x   4096  | type QuantizedDataType(groupsize=32, have_addend

In [ ]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [ ]:
# Verbose is required to pass to the callback manager
llm = GPT4All(model="./models/ggml-model-q4_0.bin", callback_manager=callback_manager, verbose=True)

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Documentation Example

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

In [ ]:
llm_chain.run(question)

 Question: What NFL team won the Super Bowl in the year Justin Bieber was born?

Answer: Let's think step by step. First, we need to identify when JB (Justin Beiber) was born - May 1st, 1988 according to his Wikipedia profile page. Then, since NFL team refers to a Football or American football team in professional National Football League league that plays during the season and is considered as one of the major sports leagues from America which has won Super Bowl on Justin Beiber's birth year; we need to look for teams winning Super Bowls before his birth.
As per Wikipedia, "The 2019 NFL Draft will be held April 25–37 in Nashville". Since JB was born in May of the same year as draft (the month and day are both zero), we can confidently conclude that he must have been alive for at least one Super Bowl win by an NFL team. However, since his birth occurred after a couple of teams winning their first ever Super Bowls such as Denver Broncos' victory in 1978 & Dallas Cowboys’ triumph over Mi

' Question: What NFL team won the Super Bowl in the year Justin Bieber was born?\n\nAnswer: Let\'s think step by step. First, we need to identify when JB (Justin Beiber) was born - May 1st, 1988 according to his Wikipedia profile page. Then, since NFL team refers to a Football or American football team in professional National Football League league that plays during the season and is considered as one of the major sports leagues from America which has won Super Bowl on Justin Beiber\'s birth year; we need to look for teams winning Super Bowls before his birth.\nAs per Wikipedia, "The 2019 NFL Draft will be held April 25–37 in Nashville". Since JB was born in May of the same year as draft (the month and day are both zero), we can confidently conclude that he must have been alive for at least one Super Bowl win by an NFL team. However, since his birth occurred after a couple of teams winning their first ever Super Bowls such as Denver Broncos\' victory in 1978 & Dallas Cowboys’ triumph 

# Poetic Example

In [ ]:
question = "Write a poem about friendship that rhymes."

In [ ]:
llm_chain.run(question)

 Question: Write a poem about friendship that rhymes.

Answer: Let's think step by step. First, let me tell you what it means to be friends with someone who is kind and caring like the sunshine on my face when we meet again after our long separation!  This could be considered as one of those rare gifts that makes life worthwhile in this world full of selfishness and dangers. Our friendship will shatter all kinds of challenges or barriers, for I believe there is always hope till the last breath has been breathed out. And we'll share tears together through good times as well as hardships like brave soldiers who are willing to take risks because they value our precious lives dearly! So friends this poem that rhymed may not be perfect but it surely says everything I have in mind for you all my life long, so please enjoy and let's cherish each other always.

" Question: Write a poem about friendship that rhymes.\n\nAnswer: Let's think step by step. First, let me tell you what it means to be friends with someone who is kind and caring like the sunshine on my face when we meet again after our long separation!  This could be considered as one of those rare gifts that makes life worthwhile in this world full of selfishness and dangers. Our friendship will shatter all kinds of challenges or barriers, for I believe there is always hope till the last breath has been breathed out. And we'll share tears together through good times as well as hardships like brave soldiers who are willing to take risks because they value our precious lives dearly! So friends this poem that rhymed may not be perfect but it surely says everything I have in mind for you all my life long, so please enjoy and let's cherish each other always."

# Mother's day Example

In [ ]:
question = "Write a social media post to celebrate mother's day."

In [ ]:
llm_chain.run(question)

 Question: Write a social media post to celebrate mother's day.

Answer: Let's think step by step. First, we need an attractive and catchy headline that will make your mom feel appreciated on this special occasion! Here are some examples of what you could write as the heading:"Mommy, You Are The Best Thing That Has Ever Happened To Me" or "I Am So Lucky to Have a Mother Like YOU".
Now it's time for an introduction. Write about your mom’s accomplishments and achievements that make her special:“My mother has the most contagious laugh I have ever heard, she can turn any ordinary day into a happy one.” or “Mommy, You are always there to listen when we need you”
Then comes the body of your post. This is where you'll share all those adorable childhood memories with her and show how much they mean to both of you:“My mom has taught me so many things that I would never have learned otherwise.” or “I can’t imagine a day without my Mommy”
In conclusion, write something heartfelt about your mother

' Question: Write a social media post to celebrate mother\'s day.\n\nAnswer: Let\'s think step by step. First, we need an attractive and catchy headline that will make your mom feel appreciated on this special occasion! Here are some examples of what you could write as the heading:"Mommy, You Are The Best Thing That Has Ever Happened To Me" or "I Am So Lucky to Have a Mother Like YOU".\nNow it\'s time for an introduction. Write about your mom’s accomplishments and achievements that make her special:“My mother has the most contagious laugh I have ever heard, she can turn any ordinary day into a happy one.” or “Mommy, You are always there to listen when we need you”\nThen comes the body of your post. This is where you\'ll share all those adorable childhood memories with her and show how much they mean to both of you:“My mom has taught me so many things that I would never have learned otherwise.” or “I can’t imagine a day without my Mommy”\nIn conclusion, write something heartfelt about y

# Explain Rain Example

In [ ]:
question = "What happens when it rains somewhere?"

In [ ]:
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's think step by step. When rain falls, first of all the water vaporizes from clouds and travel to lower altitude where air is denser. Then these drops hit surfaces like land or trees etc., which are considered as a target for this falling particles known as rainfall. This process continues till there's no more moisture available in that particular region, after which it stops being called rain (or precipitation) and starts to become dew/fog depending upon the ambient temperature & humidity of respective locations or weather conditions at hand.

" Question: What happens when it rains somewhere?\n\nAnswer: Let's think step by step. When rain falls, first of all the water vaporizes from clouds and travel to lower altitude where air is denser. Then these drops hit surfaces like land or trees etc., which are considered as a target for this falling particles known as rainfall. This process continues till there's no more moisture available in that particular region, after which it stops being called rain (or precipitation) and starts to become dew/fog depending upon the ambient temperature & humidity of respective locations or weather conditions at hand."

# Rain Example, but one sentence and funny.

In [ ]:
template = """Question: {question}

Answer: Let's answer in two sentence while being funny."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
llm_chain.run(question)

 Question: What happens when it rains somewhere?

Answer: Let's answer in two sentence while being funny. 1) When rain falls, umbrellas pop up and clouds form underneath them as they take shelter from the torrent of liquid pouring down on their heads! And...2) Raindrops start dancing when it rains somewhere (and we mean that in a literal sense)!

" Question: What happens when it rains somewhere?\n\nAnswer: Let's answer in two sentence while being funny. 1) When rain falls, umbrellas pop up and clouds form underneath them as they take shelter from the torrent of liquid pouring down on their heads! And...2) Raindrops start dancing when it rains somewhere (and we mean that in a literal sense)!"